In [1]:
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF,IDF
from pyspark.ml.feature import StringIndexer


from pyspark.ml.classification import NaiveBayes

from pyspark.ml.evaluation import MulticlassClassificationEvaluator


from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

import pandas as pd

https://spark.apache.org/docs/3.1.2/

https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.classification.NaiveBayes.html#naivebayes

https://spark.apache.org/docs/3.1.2/api/java/org/apache/spark/ml/classification/NaiveBayes.html



https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html#multiclassclassificationevaluator


https://spark.apache.org/docs/3.1.2/api/java/org/apache/spark/ml/evaluation/MulticlassClassificationEvaluator.html




In [2]:
os.environ['JAVA_HOME'] = '/opt/jdk' #Mostra aonde está o JDK

In [3]:

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "12g") \
    .appName('my-cool-app') \
    .getOrCreate()

sc=spark.sparkContext

21/10/30 08:42:01 WARN Utils: Your hostname, abner-Lenovo-G40-70 resolves to a loopback address: 127.0.1.1; using 192.168.15.20 instead (on interface wlp2s0)
21/10/30 08:42:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/30 08:42:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [5]:
caminho_df_test = 'data_test.csv'
caminho_df_training = 'data_training.csv'
#caminho_df_test = 'menordata_test.csv'
#caminho_df_training = 'menordata_training.csv'

df_test = spark.read.csv(caminho_df_test, header=True, inferSchema=True)
df_training = spark.read.csv(caminho_df_training, header=True, inferSchema=True)

In [6]:
df_test.printSchema()

root
 |-- origem: string (nullable = true)
 |-- topico: string (nullable = true)
 |-- arquivo: integer (nullable = true)
 |-- texto: string (nullable = true)



In [7]:
df_training.printSchema()

root
 |-- origem: string (nullable = true)
 |-- topico: string (nullable = true)
 |-- arquivo: integer (nullable = true)
 |-- texto: string (nullable = true)



In [8]:
label_stringIdx = StringIndexer(inputCol = "topico", outputCol = "label", handleInvalid='keep')
tokenization = Tokenizer(inputCol="texto", outputCol="palavras")
remover_stopword = StopWordsRemover(inputCol="palavras", outputCol="palavras_filtradas")
hashingTF = HashingTF(inputCol="palavras_filtradas", outputCol="tf_features")
idf = IDF(inputCol="tf_features", outputCol="tf_idf_features")

In [9]:
nb = NaiveBayes(featuresCol='tf_idf_features', labelCol='label', smoothing=1)
pipelineNB = Pipeline(stages=[label_stringIdx, tokenization, remover_stopword, hashingTF, idf, nb])

nbModel = pipelineNB.fit(df_training) #treina o modelo nb_model

21/10/30 08:42:51 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:43:02 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


In [10]:
nbparamGrid = ParamGridBuilder() \
               .addGrid(nb.smoothing, [0.0, 0.2, 0.8, 1.0]) \
               .build()

In [11]:
nbcv = CrossValidator(estimator = pipelineNB,
                      estimatorParamMaps = nbparamGrid,
                      evaluator = MulticlassClassificationEvaluator(predictionCol="prediction"),
                      numFolds = 10)

In [12]:
nbcvModel = nbcv.fit(df_training)

21/10/30 08:43:37 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:43:44 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:44:11 WARN DAGScheduler: Broadcasting large task binary with size 184.1 MiB
21/10/30 08:44:24 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:44:30 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:45:05 WARN DAGScheduler: Broadcasting large task binary with size 184.1 MiB
21/10/30 08:45:28 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:45:41 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:46:23 WARN DAGScheduler: Broadcasting large task binary with size 184.1 MiB
21/10/30 08:46:44 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:46:57 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/30 08:47:41 WARN DAGScheduler: Broadcastin

In [13]:
df_test_nbcvModel = nbcvModel.transform(df_test)

In [14]:
df_predictionAndLabels = df_test_nbcvModel.select(['prediction', 'label'])

In [15]:
labels = df_predictionAndLabels.rdd.map(lambda x: x.label).distinct().collect() # transforma o df para rdd e para poder extrair criar uma lista de labels distintas

21/10/30 09:33:24 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB


In [16]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label' )

In [17]:
vec_fMeasureByLabel = []
qtdClasses = 0
metric_list=[]

for label in sorted(labels):
    precisionByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: label})
    recallByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: label})
    fMeasureByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: label})
    
    metric_tuple_one = (label, precisionByLabel, recallByLabel,fMeasureByLabel, None , None)
    
    metric_list.append(metric_tuple_one)
    
    vec_fMeasureByLabel.append(fMeasureByLabel)
    qtdClasses +=1 

accuracy = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "accuracy"})
macroF1 = (sum(vec_fMeasureByLabel))/qtdClasses

metric_tuple_two = (None,None,None,None, accuracy, macroF1)

metric_list.append(metric_tuple_two)

21/10/30 09:33:45 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:33:59 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:34:16 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:34:31 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:34:45 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:34:59 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:35:14 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:35:29 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:35:44 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:35:57 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:36:12 WARN DAGScheduler: Broadcasting large task binary with size 186.1 MiB
21/10/30 09:36:27 WARN DAGSchedu

In [18]:
dt_metrics = pd.DataFrame(metric_list)
dt_metrics.columns=(['class', 'precision', 'recall', 'F1', 'accuracy', 'macroF1'])
dt_metrics.to_csv('metrics_naive_bayes.csv', index=False)
